In [3]:
import glob

songs = glob.glob('C:/Users/Marat/Desktop/code/machine_learning_spbu/VoiceRecognition/**/*.wav', recursive=True)

In [4]:
output_filepath = "C:\\Users\\Marat\\Desktop\\code\\machine_learning_spbu\\VoiceRecognition\\Transcipts\\"
bucketname = "maratvoicebucket"

from pydub import AudioSegment
import io
import os
from google.cloud import speech
from google.cloud import storage
from google.cloud.speech_v1 import types
import wave

In [33]:
for song in songs:
    sound = AudioSegment.from_wav(song)
    sound = sound.set_channels(1)
    sound.export(song, format="wav")

In [5]:
def frame_rate_channel(audio_file_name):
    with wave.open(audio_file_name, "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        channels = wave_file.getnchannels()
        return frame_rate,channels

In [6]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client.from_service_account_json('voice-recognition-335907-1f6a450abff9.json')
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

In [7]:
def delete_blob(bucket_name, blob_name):
    storage_client = storage.Client.from_service_account_json('voice-recognition-335907-1f6a450abff9.json')
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.delete()

In [8]:
def google_transcribe(audio_file_name):    
    frame_rate, channels = frame_rate_channel(audio_file_name)
    
    bucket_name = bucketname
    source_file_name = audio_file_name
    destination_blob_name = audio_file_name
    
    upload_blob(bucket_name, source_file_name, destination_blob_name)
    
    gcs_uri = 'gs://' + bucketname + '/' + audio_file_name
    transcript = ''
    
    client = speech.SpeechClient.from_service_account_json('voice-recognition-335907-1f6a450abff9.json')
    audio = types.RecognitionAudio(uri=gcs_uri)

    config = types.RecognitionConfig(
        encoding = speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=frame_rate,
        language_code='en-US')

    operation = client.long_running_recognize(request={"config": config, "audio": audio})
    response = operation.result(timeout=10000)

    for result in response.results:
        transcript += result.alternatives[0].transcript
    
    delete_blob(bucket_name, destination_blob_name)
    return transcript

In [9]:
def write_transcripts(transcript_filename, transcript):
    f = open(transcript_filename, "w+")
    f.write(transcript)
    f.close()

In [11]:
# for audio_file_name in songs[55:]:
audio_file_name = 'The_Pit_and_the_Pendulum_one_minute.wav'
transcript = google_transcribe(audio_file_name)
# transcript_filename = audio_file_name.split('.wav')[0] + '.txt'
# transcript_filename = transcript_filename.split('\\')[-1]
# write_transcripts(transcript_filename, transcript)
print(transcript)

The Pit and the Pendulum by Edgar Allan Poe read by Rod Harrison I was sick sick unto death with that long Agony and when they at length Unbound me and I was permitted to sit I felt that my senses were leaving me the sentence The Dread sentence of death was the last of distinct accentuation which reached my ears after that the sound of the inquisitorial voice emerged in one dreamy indeterminate hum it conveyed to my soul the idea of Revolution perhaps from its Association and fancy with the borough of a mill-wheel this only for a brief period for presently I heard no more yet for a while I saw and with how terrible and exaggerated


In [110]:
text = google_transcribe('first_times_one_minute_only_vocals.wav')

In [111]:
text

'go to feel different playing wimbley 80000 singing with me sweat up and chasing this is the dream when it was sold over grabbed a couple days just me and you then we start talking San Antonio them things left me little more'

In [107]:
from pydub import AudioSegment
t = 60 * 1000

audio = AudioSegment.from_mp3('first_times_one_minute_only_vocals.mp3')
# audio = audio[:t]
audio.export('first_times_one_minute_only_vocals.wav', format='wav')

<_io.BufferedRandom name='first_times_one_minute_only_vocals.wav'>

In [109]:
sound = AudioSegment.from_wav('first_times_one_minute_only_vocals.wav')
sound = sound.set_channels(1)
sound.export('first_times_one_minute_only_vocals.wav', format="wav")

<_io.BufferedRandom name='first_times_one_minute_only_vocals.wav'>